In [2]:
######################################################
# 1. Import libraries
######################################################

import os
import shutil
import sys
import numpy as np
from scipy.io import savemat # for saving .mat files
import wfdb

In [2]:
######################################################
# 2. Base values
######################################################

dataset_base_folder = './Grabmyo-1.0.2'
no_of_participants = 43
no_of_sessions = 3
no_of_gestures = 16
no_of_trials = 7

# Workspace values
print(f"Workspace values:")
print(f"  no_of_gestures = {no_of_gestures}")
print(f"  no_of_participants = {no_of_participants}")
print(f"  no_of_sessions = {no_of_sessions}")
print(f"  no_of_trials = {no_of_trials}")

Workspace values:
  no_of_gestures = 16
  no_of_participants = 43
  no_of_sessions = 3
  no_of_trials = 7


In [3]:
######################################################
# 3. Output folder
######################################################

output_folder = 'Converted_data'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created output folder: '{output_folder}'")
else:
    print(f"Output folder already exists.")

Created output folder: 'Converted_data'


In [ ]:
######################################################
# 4. Read files, process, and save
######################################################

count = 0

# Define channel masks for forearm and wrist data.
# The GrabMyo dataset has 32 channels.
# Forearm channels: 1-16 (0-indexed: 0-15)
forearm_channels_mask_list = [True] * 16 + [False] * 16
# Wrist channels: 18-23 and 26-31 (0-indexed: 17-22 and 25-30)
wrist_channels_mask_list = [False] * 16 + [False] * 1 + [True] * 6 + [False] * 2 + [True] * 6 + [False] * 1

# Convert lists to NumPy boolean arrays for efficient indexing, similar to MATLAB's logical().
forearm_channels_mask = np.array(forearm_channels_mask_list, dtype=bool)
wrist_channels_mask = np.array(wrist_channels_mask_list, dtype=bool)

# Each session
for session_num in range(1, no_of_sessions + 1): # MATLAB: for sessionNum = 1:noOfSessions

    current_session_raw_data_path = os.path.join(dataset_base_folder, f'Session{session_num}')   # './Grabmyo-1.0.2/Session1'

    if not os.path.exists(current_session_raw_data_path):
        print(f"Warning: Raw data folder for Session{session_num} not found at '{current_session_raw_data_path}'. Breaking the cycle.")
        break

    # Folder for converted data to be saved
    sessionNumFolderName = f'Session{session_num}_Converted' # MATLAB: ['Session', num2str(sessionNum), '_Converted']
    # MATLAB: mkdir(['Converted data' filesep sessionNumFolderName]);
    # Using output_folder variable defined earlier for consistency
    current_session_output_path = os.path.join(output_folder, sessionNumFolderName)   # ./Converted_data/Session1_Converted
    os.makedirs(current_session_output_path, exist_ok=True) # Create if not exists

    # Each participant
    for participant_num in range(1, no_of_participants + 1): # MATLAB: for participantNum = 1:noOfParticipants
        # Initialize NumPy object arrays to mimic MATLAB cell arrays
        # Shape: (no_of_trials, no_of_gestures + 1)
        # This allows storing arrays of varying sizes, similar to MATLAB's cell arrays.
        forearm_data = np.empty((no_of_trials, no_of_gestures + 1), dtype=object)
        wrist_data = np.empty((no_of_trials, no_of_gestures + 1), dtype=object)
        
        # Defining the filename to be used
        participantNumFileName = f'session{session_num}_participant{participant_num}' # session1_participant1

        participant_folder_name_in_data = f'session{session_num}_participant{participant_num}'           # 'session1_participant1'
        wfdb_record_dir = os.path.join(current_session_raw_data_path, participant_folder_name_in_data)   # ./Grabmyo-1.0.2/Session1/session1_participant1'

        if not os.path.exists(wfdb_record_dir):
            print(f"Warning: Participant folder '{participant_folder_name_in_data}' not found in '{current_session_raw_data_path}'. Breaking the cycle.")
            break

        # Each gesture for participant
        # There is a need to add an additional gesture "Resting position"
        for gesture_num in range(1, no_of_gestures + 1 + 1): # MATLAB: for gestureNum = 1:noOfGestures+1 (Add resting position)
            # Each trial for participant
            for trial_num in range(1, no_of_trials + 1): # MATLAB: for trialNum = 1:noOfTrials
                filename = f'session{session_num}_participant{participant_num}_gesture{gesture_num}_trial{trial_num}'   # 'session1_participant1_gesture1_trial1'
                filepath = os.path.join(wfdb_record_dir, filename)
                
                try:
                    # [tm, data_emg, fs, siginfo] = rdwfdb(filename);
                    # wfdb.rdrecord returns a Record object. data_emg is record.p_signal.
                    # tm (time vector) and fs (sampling frequency) are also available as record.t_signal and record.fs
                    record = wfdb.rdrecord(filepath)
                    data_emg = record.p_signal
                    # fs = record.fs # Sampling frequency (not used after reading, but available)

                    # Forearm channels are [1 - 8] and [9 - 16]
                    # Wrist channels are [18 - 23] and [26 - 31]
                    # Extracting data from specific channels related to the wrist and forearm
                    # forearmData{trialNum, gestureNum} = data_emg(:, logical(forearm_channels));
                    # wristData{trialNum, gestureNum} = data_emg(:, logical(wrist_channels));
                    forearm_data[trial_num - 1, gesture_num - 1] = data_emg[:, forearm_channels_mask]
                    wrist_data[trial_num - 1, gesture_num - 1] = data_emg[:, wrist_channels_mask]

                except Exception as e:
                    # If a specific record is not found, store it as Nonefor that entry.
                    print(f"Warning: Could not read record '{filename}' in '{wfdb_record_dir}': {e}. Breaking the cycle.")
                    forearm_data[trial_num - 1, gesture_num - 1] = None
                    wrist_data[trial_num - 1, gesture_num - 1] = None
                    break

        count += 1 # MATLAB: count = count + 1;
        print(f"Converted: {count} of {no_of_participants * no_of_sessions} participants")

        npz_file_path = os.path.join(current_session_output_path, f'{participantNumFileName}.npz')
        np.savez_compressed(npz_file_path, forearmData=forearm_data, wristData=wrist_data)
        print(f"Saved data for '{participantNumFileName}' to '{npz_file_path}'")

print('Participants file conversion completed.')

Converted: 1 of 129 participants
Saved data for 'session1_participant1' to 'Converted_data/Session1_Converted/session1_participant1.npz'
Converted: 2 of 129 participants
Saved data for 'session1_participant2' to 'Converted_data/Session1_Converted/session1_participant2.npz'
Converted: 3 of 129 participants
Saved data for 'session1_participant3' to 'Converted_data/Session1_Converted/session1_participant3.npz'
Converted: 4 of 129 participants
Saved data for 'session1_participant4' to 'Converted_data/Session1_Converted/session1_participant4.npz'
Converted: 5 of 129 participants
Saved data for 'session1_participant5' to 'Converted_data/Session1_Converted/session1_participant5.npz'
Converted: 6 of 129 participants
Saved data for 'session1_participant6' to 'Converted_data/Session1_Converted/session1_participant6.npz'
Converted: 7 of 129 participants
Saved data for 'session1_participant7' to 'Converted_data/Session1_Converted/session1_participant7.npz'
Converted: 8 of 129 participants
Saved da

In [ ]:
# CONVERTED .npz FILE STRUCTURE
#
# sessionX_participantY.npz
# │
# ├── forearmData   # shape: (7, 17), dtype=object
# │   ├── [trial 0, gesture 0] → ndarray (samples × 16 forearm channels)
# │   ├── [trial 0, gesture 1] → ndarray (...)
# │   ├── ...
# │   └── [trial 6, gesture 16] → ndarray or None if missing
# │
# └── wristData     # shape: (7, 17), dtype=object
#     ├── [trial 0, gesture 0] → ndarray (samples × 12 wrist channels)
#     ├── [trial 0, gesture 1] → ndarray (...)
#     ├── ...
#     └── [trial 6, gesture 16] → ndarray or None if missing

In [3]:
filepath1 = './Converted_data/Session3_Converted/session3_participant43.npz'
data = np.load(filepath1, allow_pickle=True)

# Extract forearm and wrist data
forearm_data = data['forearmData']
wrist_data = data['wristData']

# Access specific gesture and trial: gesture 5 (index 4), trial 7 (index 6)
selected_forearm_signal = forearm_data[6, 4]  # shape: (samples, 16)
selected_wrist_signal = wrist_data[6, 4]      # shape: (samples, 12)

# Display the loaded signal
print("Forearm signal shape:", selected_forearm_signal.shape)
print("Wrist signal shape:", selected_wrist_signal.shape)
print("Forearm sample data:\n", selected_forearm_signal)
print("Wrist sample data:\n", selected_wrist_signal)

Forearm signal shape: (10240, 16)
Wrist signal shape: (10240, 12)
Forearm sample data:
 [[-0.02875622 -0.02777223 -0.04119453 ... -0.02173353  0.0019044
  -0.0079203 ]
 [-0.03257922 -0.03051727 -0.04306322 ... -0.02202076  0.00194887
  -0.00850207]
 [-0.03690483 -0.03579621 -0.04439536 ... -0.03118468 -0.00947015
  -0.01723268]
 ...
 [-0.02102646 -0.03934566 -0.0351629  ...  0.1432854   0.06028886
   0.02855215]
 [-0.00985449  0.00302659 -0.0043202  ...  0.14752542  0.04998877
   0.01668416]
 [-0.01143852  0.02788283  0.0129791  ...  0.10412674  0.01879951
  -0.00375238]]
Wrist sample data:
 [[ 0.00564243  0.00219436 -0.00730308 ... -0.0054874  -0.00056988
   0.00580054]
 [ 0.00127648  0.00140319 -0.00914095 ... -0.00581886 -0.00080453
   0.0042831 ]
 [-0.00199797 -0.0011345  -0.01196222 ... -0.00736564 -0.00264825
   0.00293698]
 ...
 [ 0.10992558  0.02751156  0.05286271 ...  0.01959261  0.00613454
   0.01674081]
 [ 0.12785185  0.03333333  0.05971439 ...  0.01878239 -0.00771008
   0.0

In [4]:
print(data['forearmData'][0, 1].shape)
data['forearmData'][0, 1]

(10240, 16)


array([[-0.01140428,  0.01838773, -0.00531676, ...,  0.3875155 ,
         0.09664841, -0.00496973],
       [ 0.01636163,  0.07549568,  0.02005443, ...,  0.40550358,
         0.08505942,  0.03498636],
       [ 0.02649083,  0.11685935,  0.05081493, ...,  0.38312698,
         0.063767  ,  0.04232061],
       ...,
       [ 0.06831845,  0.11464677,  0.09560835, ..., -0.02601762,
         0.08043926,  0.07892107],
       [ 0.07630265,  0.11196838,  0.08573141, ..., -0.07455652,
         0.0742423 ,  0.08345188],
       [ 0.08241591,  0.10832344,  0.07463152, ..., -0.0908085 ,
         0.08009744,  0.09753988]], shape=(10240, 16))